In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import IPython
import ipywidgets as widgets

In [2]:
#SIM
##### No Policy means that nominal interest rate is always 0
##### real interest rate then = -inflation, reflecting how savings are losing money in inflation
##### this should return an inflation spiral
##### we're gonna start with inflation = 0 (piT = 0 in the code, ik it's a bit confusing)
##### also, no CBbeta because central bank doesn't exist, so inflation = output gap for all periods

###Helper Methods

# def FindOptimumY(expectedinflation, ye=100, piT=2, alpha=1, beta=1):
#     ###optimum y found using intersect of next period's PC, MR
#     #this is central bank finding its target in response to this period's distance from equilibrium
#     y = (((alpha * beta) * (piT - expectedinflation)) / (1 + ((alpha ** 2) * (beta)))) + ye
#     return y

# def FindResponse(optimumY, A, a=0.75):
#     ###optimum r using optimum y
#     #this is cb finding real interest rate response to target to get to optimal y, using its IS curve
#     r = (A - optimumY) / a 
#     return r

def InflationfromY(y, ye=100, alpha=1, beta=1, piE=0):
    ###find inflation in economy from bargaining gap 
    pi = ((ye - y) / (alpha * beta)) + piE
    return pi

###Simulator

class CEInteractivesim():
    def __init__(self, ratelist, ye=100, rstar=4, alpha=1, beta=1, a=2, 
                piT=2, credibility=0):

        self.ratelist = ratelist        
        self.periods = len(ratelist)

        self.ye = ye
        self.rstar = rstar
        self.alpha = alpha
        self.beta = beta
        self.a = a
        self.piT = piT
        self.credibility = credibility
        self.anticredibility = 1 - credibility

        self.A = ye + (a * self.rstar)
        self.cols = ['Periods', 'Output Gap', 'GDP', 'Inflation', 'Expected Inflation', 'Lending real i.r.', 'Lending nom i.r.', 
                    'A']
        
    def DemandShock(self, size, temporary=True):
        self.size = size
        self.multiplier = (0.01 * self.size) + 1
        self.newA = self.A * self.multiplier ## not sure about this
        df = pd.DataFrame(columns=self.cols)

        period = 1
        while period < 5:
            periodseries = pd.Series(dtype=np.float64)
            periodseries['Periods'] = period
            periodseries['Output Gap'] = 0.0
            periodseries['GDP'] = self.ye
            periodseries['Inflation'] = self.piT
            periodseries['Expected Inflation'] = self.piT ####up to p5, piE = piT
            cbresponsei = self.ratelist[period-1]
            periodseries['Lending nom i.r.'] = cbresponsei
            periodseries['Lending real i.r.'] = cbresponsei - self.piT
            periodseries['A'] = self.A

            df.loc[period] = periodseries
            period += 1
        
        while period < 6: #period 5 only
            #periodseries = pd.Series(dtype=np.float64)
            periodseries['Periods'] = period
            #temporary demand shock
            periodseries['GDP'] = self.ye * self.multiplier
            periodseries['Output Gap'] = self.size
            inflation = self.piT + self.size
            periodseries['Inflation'] = inflation
            periodseries['Expected Inflation'] = self.piT ###In period 5, piE = pi(t-1) = piT (for all credibility)
            #cb response, finds PC where inflation = equilibrium output (this is next period's PC)
            #then find that pc intersect with MR and that output is optimal bargaining gap
            #it uses next period's PC, so the expected inflation it uses is this period's inflation for adaptive
            #this will be useless for non-adaptive expectations - needs rewrite
            if period > self.periods:
                periodseries['Lending nom i.r.'] = np.NaN 
                periodseries['Lending real i.r.'] = np.NaN
                periodseries['A'] = np.NaN
                df.loc[period] = periodseries
                return df.round(4)


            cbresponsei = self.ratelist[period-1]
            periodseries['Lending nom i.r.'] = cbresponsei 
            periodseries['Lending real i.r.'] = cbresponsei - inflation
            periodseries['A'] = self.newA

            df.loc[period] = periodseries
            period += 1

        while (period - 1) <= self.periods: #all post shock periods
            if temporary:
                periodseries['A'] = self.A
            else:
                periodseries['A'] = self.newA
            periodseries['Periods'] = period
            #beginning of recovery
            output = periodseries['A'] - (self.a * (cbresponsei - inflation))
            periodseries['GDP'] = output
            periodseries['Output Gap'] = output - self.ye
            periodseries['Expected Inflation'] = (self.credibility * self.piT) + (self.anticredibility * inflation)
            inflation = InflationfromY(output, alpha=self.alpha, beta=self.beta, piE=periodseries['Expected Inflation'])
            periodseries['Inflation'] = inflation
            #cb response: finds PC where expected inflation = equilibrium output
            #then find that pc intersect with MR and that output is optimal bargaining gap
            #gets optimal bargaining gap with r found with RX curve
            #again, expected inflation used for next period is this period's inflation for adaptive or anchored at piT
            if period > self.periods:
                periodseries['Lending nom i.r.'] = np.NaN 
                periodseries['Lending real i.r.'] = np.NaN
                periodseries['A'] = np.NaN
                df.loc[period] = periodseries
                return df.round(4)

            cbresponsei = self.ratelist[period-1]
            periodseries['Lending nom i.r.'] = cbresponsei 
            periodseries['Lending real i.r.'] = cbresponsei - inflation
            #newq = FindQ(cbresponser)

            df.loc[period] = periodseries
            period += 1

        return df.round(4)

    def SupplyShock(self, size, temporary=True):
        self.size = size
        self.multiplier = (0.01 * self.size) + 1
        self.newye = self.ye * self.multiplier

        df = pd.DataFrame(columns=self.cols)

        period = 1
        while period < 5:
            periodseries = pd.Series(dtype=np.float64)
            periodseries['Periods'] = period
            periodseries['Output Gap'] = 0.0
            periodseries['GDP'] = self.ye
            periodseries['Inflation'] = self.piT
            periodseries['Expected Inflation'] = self.piT ####up to p5, piE = piT
            cbresponsei = self.ratelist[period-1]
            periodseries['Lending nom i.r.'] = cbresponsei
            periodseries['Lending real i.r.'] = cbresponsei - self.piT
            periodseries['A'] = self.A

            df.loc[period] = periodseries
            period += 1
        
        while period < 6:
            #periodseries = pd.Series(dtype=np.float64)
            periodseries['Periods'] = period
            #permanent supply shock changes ye, not y
            periodseries['GDP'] = self.ye
            outputgap = ((self.ye - self.newye) / self.ye) * 100
            periodseries['Output Gap'] = outputgap
            periodseries['Expected Inflation'] = self.piT 
            inflation = self.piT + outputgap
            periodseries['Inflation'] = inflation
            #cb response, finds PC where inflation = equilibrium output
            #then find that pc intersect with MR and that output is optimal bargaining gap
            #piE = df.loc[period - 1]['Inflation']
            if period > self.periods:
                periodseries['Lending nom i.r.'] = np.NaN 
                periodseries['Lending real i.r.'] = np.NaN
                periodseries['A'] = np.NaN
                df.loc[period] = periodseries
                return df.round(4)

            cbresponsei = self.ratelist[period-1]
            periodseries['Lending nom i.r.'] = cbresponsei 
            periodseries['Lending real i.r.'] = cbresponsei - inflation
            periodseries['A'] = self.A

            df.loc[period] = periodseries
            period += 1

        while (period - 1) <= self.periods:
            periodseries['Periods'] = period
            #beginning of recovery
            output = periodseries['A'] - (self.a * (cbresponsei - inflation))
            periodseries['GDP'] = output
            periodseries['Output Gap'] = output - (self.ye if temporary else self.newye)
            periodseries['Expected Inflation'] = (self.credibility * self.piT) + (self.anticredibility * inflation)
            if temporary:
                inflation = InflationfromY(output, alpha=self.alpha, beta=self.beta, piT=self.piT)
            else:
                inflation = InflationfromY(output, ye=self.newye, alpha=self.alpha, beta=self.beta, piT=self.piT)
            periodseries['Inflation'] = inflation
            #cb response, finds PC where expected inflation = equilibrium output
            #then find that pc intersect with MR and that output is optimal bargaining gap
            #piE = df.loc[period - 1]['Inflation']
            if period > self.periods:
                periodseries['Lending nom i.r.'] = np.NaN 
                periodseries['Lending real i.r.'] = np.NaN
                periodseries['A'] = np.NaN
                df.loc[period] = periodseries
                return df.round(4)
            cbresponsei = self.ratelist[period-1]
            periodseries['Lending nom i.r.'] = cbresponsei 
            periodseries['Lending real i.r.'] = cbresponsei - inflation
            periodseries['A'] = self.A

            df.loc[period] = periodseries
            period += 1
        return df.round(4)

In [3]:
#PLOT STUFF FUNCTION
from matplotlib import markers

def PlotStuff(data):
        period = int(max(data['Periods']))
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 12))

        ax1.set_xlim(1, period)
        ax2.set_xlim(1, period)
        ax3.set_xlim(1, period)
        ax4.set_xlim(1, period)

        ax1.set_ylabel('Inflation %')
        ax2.set_ylabel('GDP')
        ax3.set_ylabel('Real Interest Rate')
        ax4.set_ylabel('Nominal Interest Rate')

        ax1.set_ylim(min(data['Inflation']), max(data['Inflation']) + 2)
        ax2.set_ylim(min(data['GDP']) - 2 , max(data['GDP']) + 2)
        ax3.set_ylim(0, max(data['Lending real i.r.'][:-1]) + 2)
        ax4.set_ylim(0, max(data['Lending nom i.r.'][:-1]) + 2)

        ax1.hlines(0, xmin=1, xmax=period, color='gray')
        ax2.hlines(100, xmin=1, xmax=period, color='gray')
        ax3.hlines(4, xmin=1, xmax=period, color='gray')
        ax4.hlines(4, xmin=1, xmax=period, color='gray')

        ax1.plot(data['Periods'][:period], data['Inflation'][:period], 'red', linewidth=4, marker='.', markerfacecolor='gray', markersize=24)
        ax2.plot(data['Periods'][:period], data['GDP'][:period], 'purple', linewidth=4, marker='.', markerfacecolor='gray', markersize=24)
        ax3.plot(data['Periods'][:period-1], data['Lending real i.r.'][:period-1], 'cyan', linewidth=4, marker='.', markerfacecolor='gray', markersize=24)
        ax4.plot(data['Periods'][:period-1], data['Lending nom i.r.'][:period-1], 'blue', linewidth=4, marker='.', markerfacecolor='gray', markersize=24)

In [4]:
#output widget initialiser
out = widgets.Output(layout={'border': '2px solid gray', 'padding': '5px 5px 5px 5px', 'margin': '0 0 0 0'})
out

Output(layout=Layout(border='2px solid gray', margin='0 0 0 0', padding='5px 5px 5px 5px'))

In [7]:
#widget code and game functionality
l = [4, 4, 4, 4]
period = 4

a = widgets.FloatText(4.00, step=0.01)
sub = widgets.Button(description='Submit')
# display(a, sub)

sim = CEInteractivesim(l, a=1, piT=0)
results = sim.DemandShock(3, temporary=True)
PlotStuff(results)

with out:
    out.clear_output(wait=True)
    # print(r"There's been a 3% demand shock in period 5!")
    display(widgets.HTML(value=r"<p>There's been a 3% demand shock in period 5!</p>"))
    #print(r'Demand has increased by 3% and you need to choose a nominal rate of interest to guide the economy back to equilibrium!')
    # print(r'Choose a nominal rate with the widget below to guide next period, and click submit to see its effects')
    display(widgets.HTML(value=r"<p>Choose a nominal rate with the widget below to guide next period, and click submit to see its effects</p>"))
    # print()
    # print(r'Useful info: equilibrium nominal rate of interest = 4% and expenditure sensitivity to real interest rate (a) = 1')
    display(a, sub)
    plt.show()


def on_sub(b):
    global period
    out.clear_output(wait=True)
    l.append(a.value)
    period += 1
    #simulate using list of rates
    sim = CEInteractivesim(l, piT=0)
    results = sim.DemandShock(3, temporary=True)
    PlotStuff(results)
    with out:
        #plot stuff
        # print('Period: ', period)
        display(widgets.HTML(value=f"<p>Period: {period}</p>"))
        # print(f'You chose a nominal rate of: {a.value}%')
        display(widgets.HTML(value=f"<p>You chose a nominal rate of: {a.value}%</p>"))
        # print(f"Inflation is now: {results['Inflation'].values[-1]}%")
        display(widgets.HTML(value=f"Inflation is now: {results['Inflation'].values[-1]}%"))
        # print(f"GDP is now: {results['GDP'].values[-1]}")
        display(widgets.HTML(value=f"<p>GDP is now: {results['GDP'].values[-1]}</p>"))
        # print()
        # print('Keep going! Choose a nominal rate to affect next period and get back to equilibrium!')
        display(widgets.HTML(value=f"Keep going! Choose a nominal rate to affect next period and get back to equilibrium!</p>"))
        display(a, sub)
        # print(l)
        plt.show()

sub.on_click(on_sub)
